# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [89]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [77]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ankazobe,-18.3167,47.1167,289.66,94,100,1.92,MG,1675638367
1,1,saint-philippe,-21.3585,55.7679,298.92,86,100,4.84,RE,1675638368
2,2,bilibino,68.0546,166.4372,237.05,100,100,0.89,RU,1675638368
3,3,meulaboh,4.1363,96.1285,296.94,88,100,2.59,ID,1675638369
4,4,brigantine,39.4101,-74.3646,280.45,80,100,4.47,US,1675638123


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [78]:
%%capture --no-display

# Configure the map plot
city_data_df.hvplot(kind="scatter", x="Lng", y="Lat", size=city_data_df["Humidity"], hover_cols=["City", "Country", "Max Temp", "Current Description"], width=800, height=500, title="Weather Data", color="Max Temp", cmap="coolwarm", colorbar=True, xlim=(-180, 180), ylim=(-90, 90))
    
    
# Display the map
# YOUR CODE HERE

:Scatter   [Lng]   (Lat,Max Temp,_size,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [79]:
# Narrow down cities that fit criteria and drop any results with null values
refined_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= 300) & (city_data_df["Max Temp"] >= 250)  & (city_data_df["Cloudiness"] <= 5)].dropna()


# Display sample data
refined_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,cape town,-33.9258,18.4232,296.14,74,0,6.69,ZA,1675637776
18,18,rikitea,-23.1203,-134.9692,299.32,74,3,6.78,PF,1675638378
32,32,tarcal,48.1333,21.3500,268.99,70,2,5.92,HU,1675638386
33,33,qaanaaq,77.4840,-69.3632,254.80,55,5,2.97,GL,1675638386
34,34,bansi,27.1833,82.9333,284.18,77,0,2.65,IN,1675638387


### Step 3: Create a new DataFrame called `hotel_df`.

In [80]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = refined_cities_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
11,11,cape town,-33.9258,18.4232,296.14,74,0,6.69,ZA,1675637776,
18,18,rikitea,-23.1203,-134.9692,299.32,74,3,6.78,PF,1675638378,
32,32,tarcal,48.1333,21.3500,268.99,70,2,5.92,HU,1675638386,
33,33,qaanaaq,77.4840,-69.3632,254.80,55,5,2.97,GL,1675638386,
34,34,bansi,27.1833,82.9333,284.18,77,0,2.65,IN,1675638387,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [81]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "api_key": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cape town - nearest hotel: Townhouse Hotel
rikitea - nearest hotel: Chez Bianca & Benoit
tarcal - nearest hotel: Andrássy-kúria
qaanaaq - nearest hotel: Hotel Qaanaaq
bansi - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
hamilton - nearest hotel: North Vista Manor
punta gorda - nearest hotel: SpringHill Suites by Marriott Punta Gorda Harborside
chuy - nearest hotel: Alerces
alice springs - nearest hotel: Aurora Alice Springs
valdosta - nearest hotel: Comfort Inn & Suites
coquimbo - nearest hotel: Hotel Iberia
darab - nearest hotel: محدوده هتل جهانگردی
livada - nearest hotel: No hotel found
winslow - nearest hotel: La Posada Hotel
lebu - nearest hotel: No hotel found
salalah - nearest hotel: Muscat International Hotel
yulara - nearest hotel: Outback Pioneer Hotel & Lodge
galveston - nearest hotel: Comfort Inn & Suites Texas City
svetlaya - nearest hotel: No hotel found
mier - nearest hotel: No hotel found
barcelos - nearest hotel: Hotel Do

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
11,11,cape town,-33.9258,18.4232,296.14,74,0,6.69,ZA,1675637776,Townhouse Hotel
18,18,rikitea,-23.1203,-134.9692,299.32,74,3,6.78,PF,1675638378,Chez Bianca & Benoit
32,32,tarcal,48.1333,21.3500,268.99,70,2,5.92,HU,1675638386,Andrássy-kúria
33,33,qaanaaq,77.4840,-69.3632,254.80,55,5,2.97,GL,1675638386,Hotel Qaanaaq
34,34,bansi,27.1833,82.9333,284.18,77,0,2.65,IN,1675638387,No hotel found
...,...,...,...,...,...,...,...,...,...,...,...
543,543,sharjah,25.3573,55.4033,292.14,82,0,3.09,AE,1675638428,shakeel bhai room
546,546,tezu,27.9167,96.1667,287.93,37,1,1.77,IN,1675638666,No hotel found
552,552,rustam,27.9690,68.8048,286.12,29,0,2.35,PK,1675638669,No hotel found
553,553,lewistown,40.5992,-77.5714,282.12,48,0,1.54,US,1675638670,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [87]:
%%capture --no-display

# Configure the map plot
hotel_df_map = hotel_df.hvplot.points("Lng","Lon",geo = True, tiles="ESRI", hover_cols=["City", "Country", "Max Temp", "Current Description", "Hotel Name"], width=800, height=500, title="Weather Data", color="Max Temp", cmap="coolwarm", colorbar=True, xlim=(-180, 180), ylim=(-90, 90))

# Display the map and a world map

hotel_df_map



ImportError: Geographic projection support requires GeoViews, pyproj and cartopy.